In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run_test

import ecc
import helper
import tx
import script

In [ ]:
from tx import Tx
from io import BytesIO

stream = BytesIO(bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600'))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

In [ ]:
from helper import double_sha256

blob = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
d256 = double_sha256(blob)
z = int.from_bytes(d256, 'big')
print(hex(z))

In [ ]:
from ecc import S256Point, Signature

sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

### Exercise 1

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest::test_sig_hash
```

In [ ]:
# Exercise 1

reload(tx)
run_test(tx.TxTest('test_sig_hash'))

### Exercise 2

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest::test_verify_input
```

In [ ]:
# Exercise 2

reload(tx)
run_test(tx.TxTest('test_verify_input'))

In [ ]:
from helper import decode_base58, p2pkh_script
from script import Script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_ins.append(TxIn(prev_tx, prev_index, b'', 0xffffffff))
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
tx_outs.append(TxOut(amount=change_amount, script_pubkey=change_script))
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
tx_outs.append(TxOut(amount=target_amount, script_pubkey=target_script))
transaction = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(transaction)

In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL

hash_type = SIGHASH_ALL
z = transaction.sig_hash(0, hash_type)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + hash_type.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey

private_key = PrivateKey(secret=90210)
print(private_key.point.address(testnet=True))

### Exercise 3

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest:test_sign_input
```

In [ ]:
# Exercise 3

reload(tx)
run_test(tx.TxTest('test_sign_input'))

### Exercise 4

#### Send 40% of the tBTC you've received to this address

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`

#### Go here to send your transaction: https://live.blockcypher.com/btc-testnet/pushtx/

#### You can get some free testnet coins at: https://testnet.coinfaucet.eu/en/

In [ ]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

### Exercise 5

#### Get some more testnet coins and spend both outputs (one from your change address and one from the testnet faucet) to

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`


In [ ]:
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'